# Chapter 14 SVD

奇异值分解（Singular Value Decomposition, SVD）是一种从数据中提取主要信息的方法。

SVD能够从有噪声的数据中抽取相关特征。

SVD将原始数据矩阵分为三个部分：

${Data_{mxn} = U_{mxm}\sum_{mxn}V^T{nxn}}$

其中矩阵 ${\sum_{mxn}}$ 
- 只包含对角元素，而其他元素为0
- 对角元素从大到小排序
- 这些对角元素被称为奇异值（Singular Value）
- 奇异值是矩阵 ${Data * Data^T}$ 特征值的平方根

![](img/img1.png)

通过奇异值可以判断特征的重要性。例如，对某些数据进行SVD会发现在r个奇异值后，其余的奇异值均为0.这些0值的奇异值就是噪声或冗余特征。

## SVD工作流程

1. 准备数据（数据清洗）
2. 矩阵分解
3. 计算矩阵的能量信息
4. 重构矩阵

Numpy中的一个函数 `linalg.svd()` 可以实现矩阵的SVD


In [13]:
from numpy import *
import numpy as np
# input: array
U,Sigma,VT = linalg.svd([[1,1], [7,7]])

In [2]:
# output: array
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [3]:
Sigma

array([10.,  0.])

In [4]:
VT

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

其中 `Sigma` 只包括了对角元素，而省略了0元素。

创建如下矩阵：

```python
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]
    
```

In [17]:
import svdRec
Data = svdRec.loadExData()
U,Sigma,VT = linalg.svd(Data)
Sigma

array([9.64365076e+00, 5.29150262e+00, 7.40623935e-16, 4.05103551e-16,
       2.21838243e-32])

计算奇异值后，一种判断奇异值保留数目的方法是通过奇异值的平方和计算矩阵的能量信息。一般认为保留90%的能量信息就足够了。所以，可以通过计算矩阵能量信息来判断要保留的奇异值。


In [20]:
cumsum(Sigma**2)/sum(Sigma**2)

array([0.76859504, 1.        , 1.        , 1.        , 1.        ])

前两个奇异值相加的能量信息已经到达1（可能是由于计算误差，显示不了足够的小数）。所以，保留前两个奇异值就足够了。
接下来，根据前两个奇异值，我们可以重构矩阵

In [21]:
Sig2 = mat([[Sigma[0], 0], [0, Sigma[1]]])
U[:,:2]*Sig2*VT[:2,:]

matrix([[ 5.38896529e-16,  1.58498979e-15,  1.58498979e-15,
          2.00000000e+00,  2.00000000e+00],
        [-1.04609326e-15,  5.23046632e-16,  5.23046632e-16,
          3.00000000e+00,  3.00000000e+00],
        [-3.48697754e-16,  1.74348877e-16,  1.74348877e-16,
          1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00]])

# 相似度计算

1. 基于距离：${相似度 = 1/（1+距离）}$；距离为0时，相似度为1，距离很大时，相似度很小
2. Pearson's correlation：${0.5 + 0.5*cor}$
3. Cosine similarity: ${cos\theta = \frac{A\dot B}{||A||\dot||B||}}$ ; 两个向量方向相同，相似度为1，两个向量夹角为90度时，相似度为0

其中 ${||A||, ||B||}$ 为向量A和B的2范数，例如向量 ${[4,2,2]}$ 的2范数：${\sqrt{4^2 + 2^2 + 2^2})}$

三者的python实现：

```python
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)
```


# 小结

SVD是一种降维工具，我们可以利用SVD来逼近矩阵并从中提取重要特征。通过保留矩阵的80%~90%的能量，我们就可以提取重要的特征并去除噪声。